<a href="https://colab.research.google.com/github/PeteJoh/fpl-ai/blob/master/train_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#------------------------------------------------------
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [30]:
forward_statistics =['value', 'was_home', 'last_season_position', 'percent_value',
       'position rank', 'goals_scored_ex', 'assists_ex', 'total_points_ex',
       'minutes_ex', 'goals_conceded_ex', 'creativity_ex', 'influence_ex',
       'threat_ex', 'bonus_ex', 'bps_ex', 'ict_index_ex', 'now_cost_ex', 'GW', 'opponent_last_season_position',
        'mean assists 3','mean bonus 3', 'mean bps 3','mean creativity 3', 'mean goals_scored 3',
       'mean ict_index 3', 'mean influence 3', 'mean minutes 3', 'mean penalties_missed 3',  'mean threat 3',
       'mean total_points 3','mean value 3', 'mean match_result 3', 'std bps 3', 'std creativity 3',
       'std ict_index 3', 'std influence 3', 'std minutes 3',
       'std threat 3', 'std total_points 3', 'std value 3']

leak_columns = [
    "name",
    "team",
]  # columns that shouldnt be used in training fir fear of data leakage


dropped_columns = [
    "season",
    "opponent",
    "match_result",
   # "position",
    "assists",
    "penalties_missed",
    "bonus",
    "bps",
    "clean_sheets",
    "creativity",
    "goals_conceded",
    "goals_scored",
    "ict_index",
    "influence",
    "own_goals",
    "penalties_saved",
    "red_cards",
    "saves",
    "selected",
    "threat",
    "transfers_balance",
    "transfers_in",
    "transfers_out",
    "yellow_cards",
    "team Goal scored",
    "team Goal conceded"
]  # "value",

midfielder_statistics =['value', 'was_home', 'last_season_position', 'percent_value',
       'position rank', 'goals_scored_ex', 'assists_ex', 'total_points_ex',
       'minutes_ex', 'goals_conceded_ex', 'creativity_ex', 'influence_ex',
       'threat_ex', 'bonus_ex', 'bps_ex', 'ict_index_ex', 'now_cost_ex', 'GW', 'opponent_last_season_position',
        'mean assists 3','mean bonus 3', 'mean bps 3','mean creativity 3', 'mean goals_scored 3',
       'mean ict_index 3', 'mean influence 3', 'mean minutes 3', 'mean penalties_missed 3',  'mean threat 3',
       'mean total_points 3','mean value 3', 'mean match_result 3', 'std bps 3', 'std creativity 3',
       'std ict_index 3', 'std influence 3', 'std minutes 3',
       'std threat 3', 'std total_points 3', 'std value 3']

goalkeeper_statistics = ['value', 'was_home', 'last_season_position', 'percent_value',
       'position rank', 'total_points_ex', 'minutes_ex', 'goals_conceded_ex',
       'bonus_ex', 'bps_ex', 'ict_index_ex', 'clean_sheets_ex',
       'red_cards_ex', 'now_cost_ex', 'GW', 'opponent_last_season_position',
       'mean bonus 3', 'mean bps 3', 'mean clean_sheets 3', 'mean goals_conceded 3',
       'mean ict_index 3',  'mean minutes 3',
       'mean own_goals 3',  'mean penalties_saved 3',
        'mean saves 3',  'mean threat 3',
       'mean total_points 3',
       'mean value 3', 'mean match_result 3', 'std bps 3',
       'std ict_index 3', 'std influence 3', 'std minutes 3',
       'std threat 3', 'std total_points 3', 'std value 3']

statistics =['value', 'position','was_home', 'last_season_position', 'percent_value',
       'position rank', 'goals_scored_ex', 'assists_ex', 'total_points_ex',
       'minutes_ex', 'goals_conceded_ex', 'creativity_ex', 'influence_ex',
       'threat_ex', 'bonus_ex', 'bps_ex', 'ict_index_ex', 'clean_sheets_ex',
       'yellow_cards_ex','now_cost_ex', 'GW', 'opponent_last_season_position', 'mean assists 3',
       'mean bonus 3', 'mean bps 3', 'mean clean_sheets 3',
       'mean creativity 3', 'mean goals_conceded 3', 'mean goals_scored 3',
       'mean ict_index 3', 'mean influence 3', 'mean minutes 3',
       'mean own_goals 3',
       'mean red_cards 3',  'mean threat 3','mean total_points 3',
       'mean value 3', 'mean match_result 3', 'std bps 3', 'std creativity 3',
       'std ict_index 3', 'std influence 3', 'std minutes 3',
       'std threat 3', 'std total_points 3', 'std value 3','mean saves 3','mean assists all',
       'mean bonus all', 'mean bps all', 'mean clean_sheets all',
       'mean creativity all', 'mean goals_conceded all', 'mean goals_scored all',
       'mean ict_index all', 'mean influence all', 'mean minutes all',
       'mean own_goals all',
       'mean red_cards all',  'mean threat all','mean total_points all',
       'mean value all', 'mean match_result all',
       'mean team Goal scored 3','mean team Goal scored all','mean team Goal conceded 3','mean team Goal conceded all',"ratio_goal_scored all","ratio_goal_scored 3",
       'opp mean team Goal scored 3','opp mean team Goal conceded 3','opp mean team Goal scored all','opp mean team Goal conceded all',"opp mean match_result all"]


date_cols=["day_of week","month","hour","week"]

In [31]:
def convert_minutes(val):
    """CONVERTS MINUTES TO A CATEGORICAL OUTPUT"""
    if val > 10:
        return 1
    else:
        return 0

In [32]:
pip install catboost

In [33]:
from hyperopt import tpe,hp,fmin,STATUS_OK,Trials
from hyperopt.pyll.base import scope

In [34]:
import pandas as pd
import numpy as np
import warnings
import os


warnings.filterwarnings("ignore")
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.ensemble import (
    RandomForestClassifier,
    RandomForestRegressor,
    GradientBoostingRegressor,
)
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.metrics import (
    mean_squared_error,
    mean_absolute_error,
    confusion_matrix,
    accuracy_score,
    f1_score,
)
from lightgbm import LGBMRegressor, LGBMClassifier
from catboost import CatBoostClassifier, CatBoostRegressor
from sklearn.model_selection import StratifiedKFold,KFold

In [35]:
gameweek=10

In [36]:
train = pd.read_csv("/content/gdrive/MyDrive/fplpredict/cleaned_dataset/cleaned_previous_seasons.csv",index_col=0)#/content/drive/MyDrive/Fplpredict /cleaned_previous_seasons.csv", index_col=0)
old_gameweek_cleaned = []
for i in range(1, gameweek):
    old_gameweek_cleaned.append(pd.read_csv(f"/content/gdrive/MyDrive/fplpredict/cleaned_dataset/2024-25/GW{i}.csv"))
old_gameweeks = pd.concat(old_gameweek_cleaned)[train.columns]
train = pd.concat([train, old_gameweeks])

# data for current gameweek we want to predict on
test = pd.read_csv(f"/content/gdrive/MyDrive/fplpredict/cleaned_dataset/2024-25/GW{gameweek}.csv", index_col=0)

In [37]:
#CREATE GAMEWEEK PREDICTIONS FOLDER
path = f"/content/gdrive/MyDrive/fplpredict/predicted_dataset/GW{gameweek}"

# create new single directory
# check whether directory already exists
if not os.path.exists(path):
    os.mkdir(path)
    print("Folder %s created!" % path)
else:
    print("Folder %s already exists" % path)

# data from current season but previous gameweeks

Folder /content/gdrive/MyDrive/fplpredict/predicted_dataset/GW10 already exists


In [38]:
train["position"].value_counts()

,count
position,
MID,52571
DEF,42209
FWD,15724
GK,13166
GKP,767


In [39]:
train["position"]=train["position"].replace({"GKP":"GK" })

In [40]:
# gemini THIS CELL **********************************************************************
# Ensure both DataFrames have the same columns before alignment
# Create the date-related columns in 'test' first
import pandas as pd
from sklearn.model_selection import train_test_split

test["index"] = test["name"] + test["kickoff_time"].astype("str")
test = test.set_index("index")
test["date"] = pd.to_datetime(test["kickoff_time"], format='ISO8601', errors='coerce')
test["day_of week"] = test["date"].dt.day_name()  # Added parentheses to call the function
test["month"] = test["date"].dt.month
test["hour"] = test["date"].dt.hour
test["week"] = test["date"].dt.isocalendar().week
test.drop(["kickoff_time", "date"], axis=1, inplace=True)

# Now process 'train' DataFrame
# Create 'index' column *before* dropping 'kickoff_time'
train["index"] = train["name"] + train["kickoff_time"].astype("str")
train.drop_duplicates("index", keep="last", inplace=True)
train = train.set_index("index")

# Now you can create date features and then drop 'kickoff_time'
train["date"] = pd.to_datetime(train["kickoff_time"], format='ISO8601', errors='coerce')
train["day_of week"] = train["date"].dt.day_name()  # Added parentheses to call the function
train["month"] = train["date"].dt.month
train["hour"] = train["date"].dt.hour
train["week"] = train["date"].dt.isocalendar().week
train.drop(["kickoff_time", "date"], axis=1, inplace=True) # Now safe to drop


#Keep only common columns in both dataframes
common_cols = list(set(train.columns) & set(test.columns))

train = train[common_cols]
test = test[common_cols]



train_copy = train.copy()
test_copy = test.copy()


train["minutes"] = train["minutes"].apply(convert_minutes)

target = train[["minutes", "GW", "position"]]
train.drop(["total_points", "minutes"], axis=1, inplace=True)
test.drop(["total_points", "minutes"], axis=1, inplace=True)
train.drop(dropped_columns, axis=1, inplace=True)
test.drop(dropped_columns, axis=1, inplace=True)


# Convert categorical columns to numerical
for col in train.columns:
    if train[col].dtype == "object":
        if col not in ["team", "name", "position"]:
            train[col] = pd.factorize(train[col])[0]
            test[col] = pd.factorize(test[col])[0]

train["was_home"] = train["was_home"].replace({True: 0, False: 1})
test["was_home"] = test["was_home"].replace({True: 0, False: 1})

train = train[statistics + leak_columns + date_cols]
test = test[statistics + leak_columns + date_cols]

x, val, y, y_val = train_test_split(
    train.drop(leak_columns, axis=1),
    target["minutes"],
    test_size=0.1,
    random_state=0,
)

In [42]:
params={'colsample_bylevel': 0.8070621518153563, 'learning_rate': 0.04765984972709895, 'max_depth': 7, 'reg_lambda': 5, 'scale_pos_weight': 2.5,'subsample': 0.6794390204583894}
model=CatBoostClassifier(**params,cat_features=["position"],random_state=0,early_stopping_rounds=500,use_best_model=True,verbose=500,n_estimators=10000)

In [43]:
model.fit(x, y,eval_set=[(val,y_val)])

0:	learn: 0.6525880	test: 0.6526917	best: 0.6526917 (0)	total: 298ms	remaining: 49m 38s
500:	learn: 0.2974733	test: 0.3179153	best: 0.3179153 (500)	total: 1m 17s	remaining: 24m 28s
1000:	learn: 0.2735103	test: 0.3177685	best: 0.3176982 (972)	total: 2m 35s	remaining: 23m 20s
Stopped by overfitting detector  (500 iterations wait)

bestTest = 0.3176981932
bestIteration = 972

Shrink model to first 973 iterations.


In [44]:
print(confusion_matrix(model.predict(val), y_val))
print(
    f"starting Accuracy score {accuracy_score(model.predict(val), y_val)}"
)

print(
    f" starting f1 score: {f1_score(model.predict(val), y_val)}"
)

feature_importance = pd.DataFrame(
    {"column": x.columns, "imp": model.feature_importances_}
).sort_values(
    "imp", ascending=False
)  #

[[5495  375]
 [1401 3920]]
starting Accuracy score 0.8413010454829773
 starting f1 score: 0.8153078202995009


[[3863  267]
 [ 975 2862]]
starting Accuracy score 0.8441069411321702
 starting f1 score: 0.8217054263565893

In [45]:
feature_importance.head(50)

,column,imp
31,mean minutes 3,12.284034
56,mean minutes all,4.602828
42,std minutes 3,4.484918
29,mean ict_index 3,4.244644
20,GW,3.077469
5,position rank,3.053269
4,percent_value,2.953681
0,value,2.864990
55,mean influence all,2.834239
54,mean ict_index all,2.468474


In [46]:
feature_importance.tail(20)

,column,imp
76,hour,0.445356
69,opp mean team Goal scored 3,0.443820
8,total_points_ex,0.440845
40,std ict_index 3,0.430259
44,std total_points 3,0.393665
47,mean assists all,0.382931
70,opp mean team Goal conceded 3,0.378751
7,assists_ex,0.325190
14,bonus_ex,0.320038
58,mean red_cards all,0.308060


In [47]:
y_val

,minutes
index,
Sadio ManÃ©2021-02-07 16:30:00+00:00,1
Tim Ream2023-11-27 20:00:00+00:00,1
Chris Mepham2023-02-25 17:30:00+00:00,1
Riyad Mahrez2023-09-30 14:00:00+00:00,0
Danny Ings2020-11-01 12:00:00+00:00,1
...,...
Harvey Barnes2023-08-19 19:00:00+00:00,1
Marvelous Nakamba2021-04-21 19:15:00+00:00,1
Jean-Philippe Mateta2023-01-18 20:00:00+00:00,1


In [48]:
test_copy["minutes"] = model.predict(test.drop(leak_columns, axis=1))
test_copy[leak_columns + ["minutes"]].to_csv(
    f"minutes.csv"
)

In [49]:
predicted_minutes=model.predict(val)
val_=pd.DataFrame({"ind":val.index,"actul_minutes":y_val,"predicted_minutes":predicted_minutes,"position":val["position"]})

In [50]:
print(f"STARTING GOALKEEPERS PERFORMANCE!!!")
a=val_[val_["position"]=="GK"]["actul_minutes"]
b=val_[val_["position"]=="GK"]["predicted_minutes"]
print(confusion_matrix(a,b))
print(
    f"starting Accuracy score {accuracy_score(a,b)}"
)

print(
    f" starting f1 score: {f1_score(a,b)}"
)

STARTING GOALKEEPERS PERFORMANCE!!!
[[793  44]
 [ 27 324]]
starting Accuracy score 0.9402356902356902
 starting f1 score: 0.9012517385257302


STARTING GOALKEEPERS PERFORMANCE!!!
[[583  21]
 [ 24 216]]
starting Accuracy score 0.9466824644549763
 starting f1 score: 0.9056603773584907

STARTING DEFENDERS PERFORMANCE!!!
[[1449  208]
 [ 200  915]]
starting Accuracy score 0.8528138528138528
 starting f1 score: 0.817694369973190

STARTING MIDFIELDERS PERFORMANCE!!!
[[1595  288]
 [ 235 1195]]
starting Accuracy score 0.8421370359191066
 starting f1 score: 0.820460006865774

STARTING FORWARDS PERFORMANCE!!!
[[497  88]
 [ 69 325]]
starting Accuracy score 0.839632277834525
 starting f1 score: 0.8054522924411399

In [51]:
print(f"STARTING DEFENDERS PERFORMANCE!!!")
a=val_[val_["position"]=="DEF"]["actul_minutes"]
b=val_[val_["position"]=="DEF"]["predicted_minutes"]
print(confusion_matrix(a,b))
print(
    f"starting Accuracy score {accuracy_score(a,b)}"
)

print(
    f" starting f1 score: {f1_score(a,b)}"
)

STARTING DEFENDERS PERFORMANCE!!!
[[1843  543]
 [ 146 1282]]
starting Accuracy score 0.819349764027268
 starting f1 score: 0.7881955118352291


In [52]:
print(f"STARTING MIDFIELDERS PERFORMANCE!!!")
a=val_[val_["position"]=="MID"]["actul_minutes"]
b=val_[val_["position"]=="MID"]["predicted_minutes"]
print(confusion_matrix(a,b))
print(
    f"starting Accuracy score {accuracy_score(a,b)}"
)

print(
    f" starting f1 score: {f1_score(a,b)}"
)

STARTING MIDFIELDERS PERFORMANCE!!!
[[2185  630]
 [ 148 1821]]
starting Accuracy score 0.8373745819397993
 starting f1 score: 0.8239819004524886


In [53]:
print(f"STARTING FORWARDS PERFORMANCE!!!")
a=val_[val_["position"]=="FWD"]["actul_minutes"]
b=val_[val_["position"]=="FWD"]["predicted_minutes"]
print(confusion_matrix(a,b))
print(
    f"starting Accuracy score {accuracy_score(a,b)}"
)

print(
    f" starting f1 score: {f1_score(a,b)}"
)

STARTING FORWARDS PERFORMANCE!!!
[[674 184]
 [ 54 493]]
starting Accuracy score 0.8306049822064057
 starting f1 score: 0.8055555555555556


In [54]:
test_copy[test_copy["minutes"]==1]["position"].value_counts()

,count
position,
MID,169
DEF,114
FWD,37
GKP,24


In [55]:
train = train_copy[train_copy["minutes"] > 0]
test = test_copy[test_copy["minutes"] > 0]

In [56]:

# predict points
train["index"] = train["name"] + train["kickoff_time"].astype("str")
train.drop_duplicates("index", keep="last", inplace=True)
train= train.set_index("index")
train["date"]=pd.to_datetime(train["kickoff_time"])
train["day_of week"]=train["date"].dt.day_name
train["month"]=train["date"].dt.month
train["hour"]=train["date"].dt.hour
train["week"]=train["date"].dt.week
train.drop(["kickoff_time","date"], axis=1, inplace=True)

test["index"] = test["name"] + test["kickoff_time"].astype("str")
test = test.set_index("index")
test["date"]=pd.to_datetime(test["kickoff_time"])
test["day_of week"]=test["date"].dt.day_name
test["month"]=test["date"].dt.month
test["hour"]=test["date"].dt.hour
test["week"]=test["date"].dt.week
test.drop(["kickoff_time","date"], axis=1, inplace=True)

target = train[["total_points", "GW","position" ]]
train.drop(["total_points", "minutes"], axis=1, inplace=True)
test.drop(["total_points", "minutes"], axis=1, inplace=True)
train.drop(dropped_columns, axis=1, inplace=True)
test.drop(dropped_columns, axis=1, inplace=True)


for col in train.columns:
    if train[col].dtype == "object":
        if col not in ["team", "name","position"]:
            train[col] = pd.factorize(train[col])[0]
            test[col] = pd.factorize(test[col])[0]

train["position"]=train["position"].astype("category")
test["position"]=test["position"].astype("category")
train["was_home"] = train["was_home"].replace({True: 0, False: 1})
test["was_home"] = test["was_home"].replace({True: 0, False: 1})

train = train[statistics + leak_columns+date_cols]
test= test[statistics + leak_columns+date_cols]

x, val, y, y_val = train_test_split(
    train.drop(leak_columns, axis=1),
    target["total_points"],
    test_size=0.1,
    random_state=0,
)

KeyError: 'kickoff_time'

In [ ]:
from sklearn.model_selection import KFold

#cross_validator to splite the data into folds
folds=KFold(n_splits=8,shuffle=True,random_state=0)

#a dataframe to store the predictions made by each fold
predictions_df=pd.DataFrame()

#list to save the mean absolute errors from validatingon each folds
rmse_val=[]
rmse_X=[]

#a simple catboost regressor
model=LGBMRegressor(**{'colsample_bytree': 0.4199299182268318, 'learning_rate': 0.0032874466037521254, 'max_depth': 9, 'min_split_gain': 0.5685369160138952, 'num_leaves': 99, 'reg_alpha': 0.5621526419488447, 'reg_lambda': 0, 'subsample': 0.6534153111773866}, verbose=-50,random_state=0,early_stopping_rounds=200,n_estimators=10000)

#train model, make predictions and check the validation accuracy on  each fold
for i,(train_index,test_index) in enumerate(folds.split(train.drop(leak_columns, axis=1),target["total_points"])):
    train_fold=train.drop(leak_columns, axis=1).iloc[train_index]
    val_fold=train.drop(leak_columns, axis=1).iloc[test_index]
    y_fold=target["total_points"].iloc[train_index]
    y_val_fold=target["total_points"].iloc[test_index]


    model.fit(train_fold,y_fold,eval_set=[(val_fold,y_val_fold)])
    print(i+1)
    prediction=model.predict(test.drop(leak_columns, axis=1))
    predictions_df[i]=prediction
    rmse_val.append(mean_squared_error(model.predict(val_fold),y_val_fold,squared=False))
    rmse_X.append(mean_squared_error(model.predict(train_fold),y_fold,squared=False))
print(rmse_val)
print(rmse_X)

1
2
3
4
5
6
7
8
[2.75799904510382, 2.6652997142930492, 2.6832936632856086, 2.8264025093811855, 2.8638436004667875, 2.7048447357279364, 2.7494556057432997, 2.6377462697198215]
[1.5229543294014234, 1.9308316970996955, 1.892765585734737, 1.9056027047168056, 1.6943890843424334, 1.7993653432035002, 1.5870089580031228, 1.9578922425800178]


In [ ]:
print(np.mean(rmse_val))
print(np.mean(rmse_X))
#2.7575293874474336
#2.0174134721607295

2.7361106429651887
1.786351243135217


2.7320675301713573
1.6917027774360753

In [ ]:
predictions_df[0]

0      2.924173
1      2.924595
2      2.814342
3      2.395066
4      3.853160
         ...   
311    2.302227
312    3.181070
313    2.069684
314    1.178730
315    2.053070
Name: 0, Length: 316, dtype: float64

In [ ]:
test["points"]=np.mean(predictions_df, axis=1).values

test[leak_columns + ["points", "value"]].sort_values(
    "points", ascending=False
).to_csv("points.csv")

In [ ]:
test[test["position"]=="MID"].sort_values(by="points",ascending=False).head(11)[["name","points","team"]]

,name,points,team
index,,,
Mohamed Salah2023-10-29T14:00:00Z,Mohamed Salah,8.735140,Liverpool
Martin Ødegaard2023-10-28T14:00:00Z,Martin Ødegaard,6.533175,Arsenal
Bukayo Saka2023-10-28T14:00:00Z,Bukayo Saka,5.956754,Arsenal
Son Heung-min2023-10-27T19:00:00Z,Son Heung-min,5.859988,Spurs
Gabriel Martinelli Silva2023-10-28T14:00:00Z,Gabriel Martinelli Silva,5.840069,Arsenal
James Maddison2023-10-27T19:00:00Z,James Maddison,5.806873,Spurs
Kaoru Mitoma2023-10-29T14:00:00Z,Kaoru Mitoma,5.523091,Brighton
Bryan Mbeumo2023-10-28T11:30:00Z,Bryan Mbeumo,4.641319,Brentford
Diogo Teixeira da Silva2023-10-29T14:00:00Z,Diogo Teixeira da Silva,4.614422,Liverpool


In [ ]:
test[test["position"]=="DEF"].sort_values(by="points",ascending=False).head(10)[["name","points","team"]]

,name,points,team
index,,,
Kieran Trippier2023-10-28T16:30:00Z,Kieran Trippier,6.195080,Newcastle
Trent Alexander-Arnold2023-10-29T14:00:00Z,Trent Alexander-Arnold,5.014530,Liverpool
Lucas Digne2023-10-29T14:00:00Z,Lucas Digne,4.710693,Aston Villa
Pau Torres2023-10-29T14:00:00Z,Pau Torres,4.661043,Aston Villa
Ezri Konsa Ngoyo2023-10-29T14:00:00Z,Ezri Konsa Ngoyo,4.535804,Aston Villa
Andrew Robertson2023-10-29T14:00:00Z,Andrew Robertson,4.213994,Liverpool
Matty Cash2023-10-29T14:00:00Z,Matty Cash,4.123158,Aston Villa
Vladimír Coufal2023-10-29T13:00:00Z,Vladimír Coufal,3.972972,West Ham
Cristian Romero2023-10-27T19:00:00Z,Cristian Romero,3.945540,Spurs


In [ ]:
test[test["position"]=="GKP"].sort_values(by="points",ascending=False).head(10)[["name","points","team"]]

,name,points,team
index,,,
Emiliano Martínez Romero2023-10-29T14:00:00Z,Emiliano Martínez Romero,4.385442,Aston Villa
David Raya Martin2023-10-28T14:00:00Z,David Raya Martin,4.204943,Arsenal
Alisson Ramses Becker2023-10-29T14:00:00Z,Alisson Ramses Becker,3.967894,Liverpool
Norberto Murara Neto2023-10-28T14:00:00Z,Norberto Murara Neto,3.812593,Bournemouth
Ederson Santana de Moraes2023-10-29T15:30:00Z,Ederson Santana de Moraes,3.605701,Man City
Alphonse Areola2023-10-29T13:00:00Z,Alphonse Areola,3.600888,West Ham
Jason Steele2023-10-29T14:00:00Z,Jason Steele,3.504677,Brighton
Guglielmo Vicario2023-10-27T19:00:00Z,Guglielmo Vicario,3.356717,Spurs
Mark Flekken2023-10-28T11:30:00Z,Mark Flekken,3.235551,Brentford


In [ ]:
test[test["position"]=="FWD"].sort_values(by="points",ascending=False).head(10)[["name","points","team"]]

,name,points,team
index,,,
Erling Haaland2023-10-29T15:30:00Z,Erling Haaland,6.836103,Man City
Ollie Watkins2023-10-29T14:00:00Z,Ollie Watkins,5.577515,Aston Villa
Dominic Solanke2023-10-28T14:00:00Z,Dominic Solanke,5.046332,Bournemouth
Gabriel Fernando de Jesus2023-10-28T14:00:00Z,Gabriel Fernando de Jesus,4.544171,Arsenal
Darwin Núñez Ribeiro2023-10-29T14:00:00Z,Darwin Núñez Ribeiro,4.181486,Liverpool
Julián Álvarez2023-10-29T15:30:00Z,Julián Álvarez,3.875503,Man City
Yoane Wissa2023-10-28T11:30:00Z,Yoane Wissa,3.780475,Brentford
João Pedro Junqueira de Jesus2023-10-29T14:00:00Z,João Pedro Junqueira de Jesus,3.535017,Brighton
Callum Wilson2023-10-28T16:30:00Z,Callum Wilson,3.529454,Newcastle


In [ ]:
test["points"].sort_values(ascending=False).head(50)

index
Mohamed Salah2023-10-29T14:00:00Z                   8.735140
Erling Haaland2023-10-29T15:30:00Z                  6.836103
Martin Ødegaard2023-10-28T14:00:00Z                 6.533175
Kieran Trippier2023-10-28T16:30:00Z                 6.195080
Bukayo Saka2023-10-28T14:00:00Z                     5.956754
Son Heung-min2023-10-27T19:00:00Z                   5.859988
Gabriel Martinelli Silva2023-10-28T14:00:00Z        5.840069
James Maddison2023-10-27T19:00:00Z                  5.806873
Ollie Watkins2023-10-29T14:00:00Z                   5.577515
Kaoru Mitoma2023-10-29T14:00:00Z                    5.523091
Dominic Solanke2023-10-28T14:00:00Z                 5.046332
Trent Alexander-Arnold2023-10-29T14:00:00Z          5.014530
Lucas Digne2023-10-29T14:00:00Z                     4.710693
Pau Torres2023-10-29T14:00:00Z                      4.661043
Bryan Mbeumo2023-10-28T11:30:00Z                    4.641319
Diogo Teixeira da Silva2023-10-29T14:00:00Z         4.614422
Jarrod Bowen2023-1

In [ ]:

feature_importance = pd.DataFrame(
    {"column": x.columns, "imp": model.feature_importances_}
).sort_values(
    "imp", ascending=False
)  #

In [ ]:
feature_importance.head(50)

,column,imp
71,opp mean team Goal scored all,13260
72,opp mean team Goal conceded all,12240
73,opp mean match_result all,12043
64,mean team Goal scored all,10330
62,mean match_result all,10148
66,mean team Goal conceded all,9773
4,percent_value,9079
30,mean influence 3,7914
41,std influence 3,7710
40,std ict_index 3,7637


In [ ]:
feature_importance.tail(30)

,column,imp
47,mean assists all,4606
13,threat_ex,4594
12,influence_ex,4489
35,mean total_points 3,4473
16,ict_index_ex,4352
65,mean team Goal conceded 3,4266
53,mean goals_scored all,3896
37,mean match_result 3,3873
27,mean goals_conceded 3,3276
14,bonus_ex,3126


#Save predictions

In [ ]:
test_copy[test_copy["position"]=="DEF"][["name","team","minutes"]].to_csv(f"/content/gdrive/MyDrive/fplpredict/predicted_dataset/GW{gameweek}/defenders_minutes.csv")
test_copy[test_copy["position"]=="GKP"][["name","team","minutes"]].to_csv(f"/content/gdrive/MyDrive/fplpredict/predicted_dataset/GW{gameweek}/goalkeepers_minutes.csv")
test_copy[test_copy["position"]=="MID"][["name","team","minutes"]].to_csv(f"/content/gdrive/MyDrive/fplpredict/predicted_dataset/GW{gameweek}/midfielders_minutes.csv")
test_copy[test_copy["position"]=="FWD"][["name","team","minutes"]].to_csv(f"/content/gdrive/MyDrive/fplpredict/predicted_dataset/GW{gameweek}/forwards_minutes.csv")

In [ ]:
test[test["position"]=="DEF"][["name","team","points","value"]].to_csv(f"/content/gdrive/MyDrive/fplpredict/predicted_dataset/GW{gameweek}/defenders_points.csv")
test[test["position"]=="GKP"][["name","team","points","value"]].to_csv(f"/content/gdrive/MyDrive/fplpredict/predicted_dataset/GW{gameweek}/goalkeepers_points.csv")
test[test["position"]=="MID"][["name","team","points","value"]].to_csv(f"/content/gdrive/MyDrive/fplpredict/predicted_dataset/GW{gameweek}/midfielders_points.csv")
test[test["position"]=="FWD"][["name","team","points","value"]].to_csv(f"/content/gdrive/MyDrive/fplpredict/predicted_dataset/GW{gameweek}/forwards_points.csv")